# Text Classification Model for IMDB Movie Sentiment Using TensorFlow Take 2
### David Lowe
### April 2, 2021

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: This project aims to construct a text classification model using a neural network and document the end-to-end steps using a template. The IMDB Movie Sentiment dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: This dataset contains 50,000 movie reviews extracted  from IMDB. The researchers have annotated the tweets with labels (0 = negative, 1 = positive) to detect sentiment from the reviews.

From iteration Take1, we created a bag-of-words model to perform binary classification (positive or negative) for the Tweets. The Part A script focused on building the model with the training and validation datasets due to memory capacity constraints. Part B focused on testing the model with the training and test datasets.

In this Take2 iteration, we will create a word-embedding model to perform binary classification for the Tweets.

ANALYSIS: From iteration Take1, the preliminary model's performance achieved an accuracy score of 88.80% on the validation dataset after 10 epochs. Furthermore, the final model processed the test dataset with an accuracy measurement of 89.48%.

In this Take2 iteration, the preliminary model's performance achieved an average accuracy score of 88.40% on the validation dataset after 10 epochs. Furthermore, the final model processed the test dataset with an accuracy measurement of 89.66%.

CONCLUSION: In this iteration, the word-embedding TensorFlow model appeared to be suitable for modeling this dataset. We should consider experimenting with TensorFlow for further modeling.

Dataset Used: IMDB Movie Sentiment

Dataset ML Model: Binary class text classification with text-oriented features

Dataset Reference: https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format

One potential source of performance benchmarks: https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format

A deep-learning text classification project generally can be broken down into five major tasks:

1. Prepare Environment
2. Load and Prepare Text Data
3. Define and Train Models
4. Evaluate and Optimize Models
5. Finalize Model and Make Predictions

# Task 1 - Prepare Environment

In [1]:
# # Install the packages to support accessing environment variable and SQL databases
# !pip install python-dotenv PyMySQL boto3

In [2]:
# # Retrieve GPU configuration information from Colab
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#     print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
#     print('and then re-execute this cell.')
# else:
#     print(gpu_info)

In [3]:
# # Retrieve memory configuration information from Colab
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#     print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
#     print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#     print('re-execute this cell.')
# else:
#     print('You are using a high-RAM runtime!')

In [4]:
# Retrieve CPU information from the system
ncpu = !nproc
print("The number of available CPUs is:", ncpu[0])

The number of available CPUs is: 4


## 1.a) Load libraries and modules

In [5]:
# Set the random seed number for reproducible results
seedNum = 888

In [6]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
import shutil
import string
import nltk
# import boto3
# from dotenv import load_dotenv
from nltk.corpus import stopwords
from collections import Counter
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
tf.random.set_seed(seedNum)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [7]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Package movie_reviews is a

True

## 1.b) Set up the controlling parameters and functions

In [8]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the number of CPU cores available for multi-thread processing
n_jobs = 1

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the verbose level for program execution output
verbose = False

# Set Pandas options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Set the percentage sizes for splitting the dataset
TEST_SET_SIZE = 0.2
VAL_SET_SIZE = 0.25

# Set the number of folds for cross validation
N_FOLDS = 5
N_ITERATIONS = 1

# Set various default modeling parameters
DEFAULT_LOSS = 'binary_crossentropy'
DEFAULT_METRICS = ['accuracy']
DEFAULT_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)
DEFAULT_INITIALIZER = tf.keras.initializers.RandomNormal(seed=seedNum)
CLASSIFIER_ACTIVATION = 'sigmoid'
MAX_EPOCH = 10
BATCH_SIZE = 64
MIN_OCCURANCE = 1
NUM_CLASSES = 1
DEFAULT_VECTOR_SPACE = 128
DEFAULT_FILTERS = 32
DEFAULT_KERNEL_SIZE = 16
DEFAULT_POOL_SIZE = 2

# Define the labels to use for graphing the data
train_metric = "accuracy"
validation_metric = "val_accuracy"
train_loss = "loss"
validation_loss = "val_loss"

# Check the number of GPUs accessible through TensorFlow
print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))

# Print out the TensorFlow version for confirmation
print('TensorFlow version:', tf.__version__)

Num GPUs Available: 0
TensorFlow version: 2.3.1


In [9]:
# Set up the parent directory location for loading the dotenv files
# from google.colab import drive
# drive.mount('/content/gdrive')
# gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
# env_path = '/content/gdrive/My Drive/Colab Notebooks/'
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

# Set up the dotenv file for retrieving environment variables
# env_path = "/Users/david/PycharmProjects/"
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

In [10]:
# Set up the email notification function
def status_notify(msg_text):
    access_key = os.environ.get('SNS_ACCESS_KEY')
    secret_key = os.environ.get('SNS_SECRET_KEY')
    aws_region = os.environ.get('SNS_AWS_REGION')
    topic_arn = os.environ.get('SNS_TOPIC_ARN')
    if (access_key is None) or (secret_key is None) or (aws_region is None):
        sys.exit("Incomplete notification setup info. Script Processing Aborted!!!")
    sns = boto3.client('sns', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=aws_region)
    response = sns.publish(TopicArn=topic_arn, Message=msg_text)
    if response['ResponseMetadata']['HTTPStatusCode'] != 200 :
        print('Status notification not OK with HTTP status code:', response['ResponseMetadata']['HTTPStatusCode'])

In [11]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [12]:
# Reset the random number generators
def reset_random(x):
    random.seed(x)
    np.random.seed(x)
    tf.random.set_seed(x)

In [13]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 2 - Load and Prepare Text Data

In [14]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 2.a) Download Text Data Archive

In [15]:
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-imdb-movie-sentiment/Train.csv'
Xy_train = pd.read_csv(dataset_path)

# Take a peek at the dataframe after import
print(Xy_train.info())
print(Xy_train.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB
None
                                                text  label
0  I grew up (b. 1965) watching and loving the Th...      0
1  When I put this movie in my DVD player, and sa...      0
2  Why do people who do not know what a particula...      0
3  Even though I have great interest in Biblical ...      0
4  Im a die hard Dads Army fan and nothing will e...      1
5  A terrible movie as everyone has said. What ma...      0
6  Finally watched this shocking movie last night...      1
7  I caught this film on AZN on cable. It sounded...      0
8  It may be the remake of 1987 Autumn's Tale aft...      1
9  My Super Ex Girlfriend turned out to be a plea...      1


In [16]:
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-imdb-movie-sentiment/Valid.csv'
Xy_valid = pd.read_csv(dataset_path)

# Take a peek at the dataframe after import
print(Xy_valid.info())
print(Xy_valid.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB
None
                                                text  label
0  It's been about 14 years since Sharon Stone aw...      0
1  someone needed to make a car payment... this i...      0
2  The Guidelines state that a comment must conta...      0
3  This movie is a muddled mish-mash of clichés f...      0
4  Before Stan Laurel became the smaller half of ...      0
5  This is the best movie I've ever seen! <br /><...      1
6  The morbid Catholic writer Gerard Reve (Jeroen...      1
7  "Semana Santa" or "Angel Of Death" is a very w...      0
8  Somebody mastered the difficult task of mergin...      1
9  Why did I waste 1.5 hours of my life watching ...      0


## 2.b) Splitting Data into Feature and Label Sets

In [17]:
X_train_df = Xy_train.iloc[:,0]
y_train_df = Xy_train.iloc[:,1]
print('X_train_df.shape: {} y_train_df.shape: {}'.format(X_train_df.shape, y_train_df.shape))

X_train_df.shape: (40000,) y_train_df.shape: (40000,)


In [18]:
X_valid_df = Xy_valid.iloc[:,0]
y_valid_df = Xy_valid.iloc[:,1]
print('X_valid_df.shape: {} y_valid_df.shape: {}'.format(X_valid_df.shape, y_valid_df.shape))

X_valid_df.shape: (5000,) y_valid_df.shape: (5000,)


In [19]:
X_complete_df = pd.concat([X_train_df, X_valid_df], ignore_index=True)
y_complete_df = pd.concat([y_train_df, y_valid_df], ignore_index=True)
print('X_complete_df.shape: {} y_complete_df.shape: {}'.format(X_complete_df.shape, y_complete_df.shape))

X_complete_df.shape: (45000,) y_complete_df.shape: (45000,)


## 2.c) Load Document and Build Vocabulary

In [20]:
# turn a sentence into clean tokens
def clean_sentence(sentence):
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [21]:
# load sentences and add to vocab
def add_sentence_to_vocab(sentence, vocab):
    # clean doc
    tokens = clean_sentence(sentence)
    # update counts
    vocab.update(tokens)

In [22]:
# load all sentences in the dataframe
def build_vocabulary(comments, vocab):
    # walk through all comments in the dataframe
    for i in range(len(comments)):
        # add comments to vocab
        sentence = comments.iloc[i]
        add_sentence_to_vocab(sentence, vocab)
        if verbose : print('Processing comment:', sentence)
        if ((i+1) % 100000) == 0 : print(i+1, 'comments processed so far.')
    print('Total number of comments loaded into the vocabulary:', i+1, '\n')

In [23]:
# define vocab
vocab = Counter()
# add all docs to vocab
build_vocabulary(X_complete_df, vocab)
# print the size of the vocab
print('The total number of words in the vocabulary:', len(vocab))
# print the top words in the vocab
top_words = 50
print('The top', top_words, 'words in the vocabulary:\n', vocab.most_common(top_words))

Total number of comments loaded into the vocabulary: 45000 

The total number of words in the vocabulary: 195713
The top 50 words in the vocabulary:
 [('br', 102476), ('The', 79702), ('movie', 74201), ('film', 65917), ('one', 41670), ('like', 33799), ('This', 26243), ('good', 24690), ('It', 21382), ('would', 21355), ('time', 20444), ('really', 20025), ('see', 19639), ('even', 19348), ('story', 19257), ('much', 16539), ('get', 16019), ('people', 15037), ('bad', 14970), ('great', 14847), ('well', 14162), ('first', 13793), ('also', 13724), ('make', 13676), ('made', 13654), ('movies', 13619), ('films', 13560), ('could', 13413), ('way', 13161), ('dont', 13047), ('characters', 12999), ('But', 12592), ('think', 12516), ('And', 11900), ('Its', 11823), ('seen', 11695), ('character', 11505), ('watch', 11271), ('many', 11235), ('never', 10931), ('acting', 10908), ('know', 10907), ('plot', 10833), ('two', 10802), ('In', 10515), ('little', 10469), ('love', 10403), ('show', 10341), ('best', 10190), 

In [24]:
# keep tokens with a min occurrence in the vocabulary
tokens = [k for k,c in vocab.items() if c >= MIN_OCCURANCE]
print('The number of words with the minimum appearance:', len(tokens))

The number of words with the minimum appearance: 195713


In [25]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocabulary.txt')

## 2.d) Create Tokenizer and Encode the Input Text

In [26]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [27]:
# load the vocabulary
vocab_filename = 'vocabulary.txt'
vocab_text = load_doc(vocab_filename)
vocab_words = vocab_text.split()
vocab = set(vocab_words)
print('Number of tokens in the vocabulary:', len(vocab))

Number of tokens in the vocabulary: 195713


In [28]:
# load comments, clean and return line of tokens
def comment_to_line(sentence, vocab):
    # clean sentence
    tokens = clean_sentence(sentence)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    line = ' '.join(tokens)
    return line

In [29]:
# load all comments in a directory
def process_comments_to_lines(comments, vocab):
    lines = list()
    # walk through all comments in the dataframe
    for i in range(len(comments)):
        # load and clean the comments
        sentence = comments.iloc[i]
        line = comment_to_line(sentence, vocab)
        # add to list
        lines.append(line)
    return lines

In [30]:
# Load all training cases
train_docs = process_comments_to_lines(X_train_df, vocab)
print('Number of comments processed:', len(train_docs))

Number of comments processed: 40000


In [31]:
# Load all validation cases
valid_docs = process_comments_to_lines(X_valid_df, vocab)
print('Number of comments processed:', len(valid_docs))

Number of comments processed: 5000


In [32]:
# prepare word-embedding encoding of training dataset
def encode_train_data(docs_set, maxlen):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(docs_set)
    # encode training data set
    encoded_docs = tokenizer.texts_to_sequences(docs_set)
    # pad sequences
    train_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    # define vocabulary size (largest integer value)
    vocab_size = len(tokenizer.word_index) + 1
    return train_encoded, tokenizer, vocab_size

In [33]:
# prepare word-embedding encoding of test dataset
def encode_valid_data(docs_set, tokenizer, maxlen):
    # encode validation data set
    encoded_docs = tokenizer.texts_to_sequences(docs_set)
    # pad sequences
    val_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    return val_encoded

In [34]:
# Get maximum doc length for padding sequences
max_length = max([len(s.split()) for s in train_docs])
print('The maximum document length:', max_length)

The maximum document length: 1480


In [35]:
# Encode the training dataset
X_train, trained_tokenizer, vocabulary_size = encode_train_data(train_docs, max_length)
print('The size of encoded vocabulary:', vocabulary_size)
print('The shape of the encoded training dataset:', X_train.shape)

The size of encoded vocabulary: 154729
The shape of the encoded training dataset: (40000, 1480)


In [36]:
y_train = y_train_df.values.ravel()
print('The shape of the encoded test classes:', y_train.shape)

The shape of the encoded test classes: (40000,)


In [37]:
# Encode the validation dataset
X_valid = encode_valid_data(valid_docs, trained_tokenizer, max_length)
print('The shape of the encoded validation dataset:', X_valid.shape)

The shape of the encoded validation dataset: (5000, 1480)


In [38]:
y_valid = y_valid_df.values.ravel()
print('The shape of the encoded training classes:', y_train.shape)

The shape of the encoded training classes: (40000,)


In [39]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 3 - Define and Train Models

In [40]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [41]:
# Define the baseline model for benchmarking
def create_nn_model(input_param=max_length, output_param=NUM_CLASSES, n_vocab=vocabulary_size, n_vectors=DEFAULT_VECTOR_SPACE, n_pools=DEFAULT_POOL_SIZE, 
                    conv1_filters=DEFAULT_FILTERS, conv1_kernels=DEFAULT_KERNEL_SIZE, dense_nodes=64, opt_param=DEFAULT_OPTIMIZER):
    nn_model = keras.Sequential([
        layers.Embedding(n_vocab, n_vectors, input_length=input_param),
        layers.Conv1D(filters=conv1_filters, kernel_size=conv1_kernels, activation='relu', kernel_initializer=DEFAULT_INITIALIZER),
        layers.MaxPooling1D(pool_size=n_pools),
        layers.Flatten(),
        layers.Dense(dense_nodes, activation='relu', kernel_initializer=DEFAULT_INITIALIZER),
        layers.Dense(output_param, activation=CLASSIFIER_ACTIVATION)
    ])
    nn_model.compile(loss=DEFAULT_LOSS, optimizer=opt_param, metrics=DEFAULT_METRICS)
    return nn_model

In [42]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
reset_random(seedNum)
baseline_model = create_nn_model()
baseline_model_history = baseline_model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCH,
	validation_data=(X_valid, y_valid),
    verbose=1)
print('Total time for model fitting:', (datetime.now() - startTimeModule))

Epoch 1/10
625/625 [==============================] - 1014s 2s/step - loss: 0.3418 - accuracy: 0.8345 - val_loss: 0.2495 - val_accuracy: 0.8946
Epoch 2/10
625/625 [==============================] - 1005s 2s/step - loss: 0.0683 - accuracy: 0.9771 - val_loss: 0.3317 - val_accuracy: 0.8844
Epoch 3/10
625/625 [==============================] - 1005s 2s/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.4859 - val_accuracy: 0.8834
Epoch 4/10
625/625 [==============================] - 994s 2s/step - loss: 1.3641e-04 - accuracy: 1.0000 - val_loss: 0.5385 - val_accuracy: 0.8842
Epoch 5/10
625/625 [==============================] - 996s 2s/step - loss: 4.6467e-05 - accuracy: 1.0000 - val_loss: 0.5997 - val_accuracy: 0.8836
Epoch 6/10
625/625 [==============================] - 995s 2s/step - loss: 2.6681e-05 - accuracy: 1.0000 - val_loss: 0.6302 - val_accuracy: 0.8852
Epoch 7/10
625/625 [==============================] - 991s 2s/step - loss: 1.5004e-05 - accuracy: 1.0000 - val_loss: 0.6424 - v

In [43]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 4 - Evaluate and Optimize Models

In [44]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [45]:
# Not applicable for this iteration of modeling

In [46]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 5 - Finalize Model and Make Predictions

In [47]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 5.a) Train the Final Model

In [48]:
# Load all training cases
complete_docs = process_comments_to_lines(X_complete_df, vocab)
print('Number of comments processed:', len(complete_docs))

Number of comments processed: 45000


In [49]:
# Get maximum doc length for padding sequences
final_max_length = max([len(s.split()) for s in train_docs])
print('The maximum document length:', final_max_length)

The maximum document length: 1480


In [50]:
# Encode the training dataset
X_complete, final_tokenizer, final_vocabulary_size = encode_train_data(complete_docs, final_max_length)
print('The size of encoded vocabulary:', final_vocabulary_size)
print('The shape of the encoded final dataset:', X_complete.shape)

The size of encoded vocabulary: 165150
The shape of the encoded final dataset: (45000, 1480)


In [51]:
y_complete = y_complete_df.values.ravel()
print('The shape of the encoded training classes:', y_complete.shape)

The shape of the encoded training classes: (45000,)


In [52]:
# Create the final model for evaluating the test dataset
FINAL_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=0.001)
reset_random(seedNum)
final_model = create_nn_model(input_param=final_max_length, n_vocab=final_vocabulary_size, opt_param=FINAL_OPTIMIZER)
final_model.fit(X_complete, y_complete, epochs=MAX_EPOCH, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/10
704/704 [==============================] - 1130s 2s/step - loss: 0.3409 - accuracy: 0.8384
Epoch 2/10
704/704 [==============================] - 1136s 2s/step - loss: 0.0695 - accuracy: 0.9766
Epoch 3/10
704/704 [==============================] - 1147s 2s/step - loss: 0.0041 - accuracy: 0.9991
Epoch 4/10
704/704 [==============================] - 1123s 2s/step - loss: 2.1416e-04 - accuracy: 1.0000
Epoch 5/10
704/704 [==============================] - 1131s 2s/step - loss: 4.8620e-05 - accuracy: 1.0000
Epoch 6/10
704/704 [==============================] - 1139s 2s/step - loss: 2.4847e-05 - accuracy: 1.0000
Epoch 7/10
704/704 [==============================] - 1145s 2s/step - loss: 1.4215e-05 - accuracy: 1.0000
Epoch 8/10
704/704 [==============================] - 1157s 2s/step - loss: 8.4861e-06 - accuracy: 1.0000
Epoch 9/10
704/704 [==============================] - 1141s 2s/step - loss: 5.2091e-06 - accuracy: 1.0000
Epoch 10/10
704/704 [==============================] - 113

In [53]:
# Summarize the final model
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1480, 128)         21139200  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1465, 32)          65568     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 732, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 23424)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1499200   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 22,704,033
Trainable params: 22,704,033
Non-trainable params: 0
__________________________________________

## 5.b) Load Test Dataset and Make Predictions

In [54]:
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-imdb-movie-sentiment/Test.csv'
Xy_test = pd.read_csv(dataset_path)

# Take a peek at the dataframe after import
print(Xy_test.info())
print(Xy_test.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB
None
                                                text  label
0  I always wrote this series off as being a comp...      0
1  1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...      0
2  This movie was so poorly written and directed ...      0
3  The most interesting thing about Miryang (Secr...      1
4  when i first read about "berlin am meer" i did...      0
5  I saw this film on September 1st, 2005 in Indi...      1
6  I saw a screening of this movie last night. I ...      0
7  William Hurt may not be an American matinee id...      1
8  IT IS A PIECE OF CRAP! not funny at all. durin...      0
9  I'M BOUT IT(1997)<br /><br />Developed & publi...      0


In [55]:
X_test_df = Xy_test.iloc[:,0]
y_test_df = Xy_test.iloc[:,1]
print("X_test_df.shape: {} y_test_df.shape: {}".format(X_test_df.shape, y_test_df.shape))

X_test_df.shape: (5000,) y_test_df.shape: (5000,)


In [56]:
# Load all test cases
test_docs = process_comments_to_lines(X_test_df, vocab)
print('Number of comments processed:', len(test_docs))

Number of comments processed: 5000


In [57]:
# prepare word-embedding encoding of test dataset
def encode_test_data(docs_set, tokenizer, maxlen):
    # encode validation data set
    encoded_docs = tokenizer.texts_to_sequences(docs_set)
    # pad sequences
    val_encoded = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
    return val_encoded

In [58]:
# encode test dataset
X_test = encode_test_data(test_docs, final_tokenizer, final_max_length)
print('The shape of the encoded test dataset:', X_test.shape)

The shape of the encoded test dataset: (5000, 1480)


In [59]:
y_test = y_test_df.values.ravel()
print('The shape of the encoded testing classes:', y_test.shape)

The shape of the encoded testing classes: (5000,)


In [60]:
# test_predictions = final_model.predict(X_test, batch_size=default_batch, verbose=1)
test_predictions = (final_model.predict(X_test) > 0.5).astype("int32").ravel()
print('Accuracy Score:', accuracy_score(y_test, test_predictions))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Accuracy Score: 0.8966
[[2230  265]
 [ 252 2253]]
              precision    recall  f1-score   support

           0       0.90      0.89      0.90      2495
           1       0.89      0.90      0.90      2505

    accuracy                           0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000



In [61]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [62]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 6:02:33.891198
